<a href="https://colab.research.google.com/github/fatemehrashidi7/Shabake_Mokhaberati/blob/main/Golay_codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 💻 شبیه‌سازی کدهای گلی (Golay Codes)
# این کد از قابلیت تصحیح خطای کد گلی G24 برای تخمین نرخ خطای پس از رمزگشایی استفاده می‌کند.

import numpy as np
import random

# --- ۱. تعریف پارامترهای کد گلی ---
n = 24  # طول کد (Code length)
k = 12  # طول پیام (Message length)
t = 3   # حداکثر تعداد خطای قابل تصحیح (Error-correcting capability)

# --- ۲. توابع شبیه‌سازی ---

def generate_message(k):
    """تولید یک پیام تصادفی k بیتی."""
    return np.random.randint(0, 2, size=k)

def golay_encode_mock(message):
    """تابع ساختگی برای کدگذاری گلی (برای حفظ طول n=24)."""
    if len(message) != k:
        raise ValueError(f"طول پیام باید {k} باشد.")
    # تولید 12 بیت اضافی (ساختگی) برای رسیدن به طول 24
    mock_parity = np.zeros(n - k, dtype=int)
    return np.concatenate((message, mock_parity)) #

def binary_symmetric_channel(codeword, p_error):
    """
    شبیه‌سازی کانال متقارن دوتایی (BSC).
    با احتمال p_error هر بیت را معکوس می‌کند.
    """
    errors = np.random.rand(n) < p_error
    # خطاهای XOR: (0 + 1) % 2 = 1, (1 + 1) % 2 = 0
    received_word = (codeword + errors) % 2

    num_errors = np.sum(errors)
    return received_word, num_errors #

def golay_decode_mock(received_word, num_errors):
    """
    تابع ساختگی برای رمزگشایی گلی: بر اساس آستانه تصحیح خطا (t=3) تصمیم می‌گیرد.
    """
    if num_errors <= t:
        # اگر تعداد خطاها کمتر یا مساوی t باشد، تصحیح موفق است.
        is_successful = True
    else:
        # اگر تعداد خطاها بیشتر از t باشد، تصحیح شکست می‌خورد.
        is_successful = False

    # بخش پیام (k بیت اول) را برمی‌گردانیم
    decoded_message = received_word[:k]

    return decoded_message, is_successful

# --- ۳. پارامترهای اجرا و شروع شبیه‌سازی ---
num_simulations = 100000  # تعداد کلمه کدی که شبیه‌سازی می‌کنیم
p_error_channel = 0.01   # احتمال خطای بیت در کانال (می‌توانید این را تغییر دهید)

# --- ۴. اجرای حلقه شبیه‌سازی ---
total_errors_introduced = 0
successful_decodes = 0
failed_decodes = 0
bit_errors_after_decode = 0
total_bits_transmitted = num_simulations * k

print(f"آغاز شبیه‌سازی {num_simulations} کلمه کد G24 با p_error={p_error_channel}...")
print("-" * 50)

for _ in range(num_simulations):
    # الف) تولید و کدگذاری
    message = generate_message(k)
    codeword = golay_encode_mock(message)

    # ب) کانال نویزی
    received_word, num_errors_introduced = binary_symmetric_channel(codeword, p_error_channel)
    total_errors_introduced += num_errors_introduced

    # ج) رمزگشایی
    decoded_message, is_successful = golay_decode_mock(received_word, num_errors_introduced)

    # د) ارزیابی
    if is_successful:
        successful_decodes += 1
    else:
        failed_decodes += 1
        # محاسبه خطاهای باقیمانده (فقط در صورت شکست تصحیح)
        bit_errors_after_decode += np.sum((message != decoded_message))

# --- ۵. نتایج ---
print("\n--- نتایج شبیه‌سازی ---")
print(f"تعداد کل کلمه کدهای شبیه‌سازی شده: {num_simulations}")
print(f"احتمال خطای بیت در کانال (p_error): {p_error_channel}")

# BER کانال (قبل از تصحیح)
channel_ber = total_errors_introduced / (num_simulations * n)
print(f"نرخ خطای بیت (BER) کانال (بدون کد): {channel_ber:.5e}")

# نرخ خطای کلمه
word_error_rate = failed_decodes / num_simulations
print(f"نرخ خطای کلمه (Word Error Rate - WER): {word_error_rate:.5e}")

# BER پس از رمزگشایی
if total_bits_transmitted > 0:
    post_decode_ber = bit_errors_after_decode / total_bits_transmitted
    print(f"نرخ خطای بیت (BER) پس از رمزگشایی: {post_decode_ber:.5e}")
else:
    print("خطا در محاسبه BER پس از رمزگشایی.")

آغاز شبیه‌سازی 100000 کلمه کد G24 با p_error=0.01...
--------------------------------------------------

--- نتایج شبیه‌سازی ---
تعداد کل کلمه کدهای شبیه‌سازی شده: 100000
احتمال خطای بیت در کانال (p_error): 0.01
نرخ خطای بیت (BER) کانال (بدون کد): 1.00054e-02
نرخ خطای کلمه (Word Error Rate - WER): 1.40000e-04
نرخ خطای بیت (BER) پس از رمزگشایی: 2.41667e-05
